In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
sys.path.append("../../tools")
from plot_tools import plot_uv_map,plot_map_zoom

import netCDF4 as nc

In [2]:
# Import reference grid 
map_4th = "/Odyssey/private/t22picar/data/glorys_15m/glorys_multivar_15m_2010-2018.nc"
map_4th = xr.open_dataset(map_4th).sel(time="2010-01-01")
lat_ref = map_4th.lat.values
lon_ref = map_4th.lon.values

In [3]:
# Import data to interp

file_map = "/Odyssey/private/t22picar/data/ssh_L4/SSH_L4_CMEMS_2010-01-01-2019-01-01.nc"
map = xr.open_dataset(file_map).sel(time=slice("2010-01-01","2010-01-01"))

map = map.rename({"longitude" : "lon"})
map = map.rename({"latitude" : "lat"})
map = map.rename({"adt" : "zos"})
#map

map = map.interp({"lat":lat_ref, "lon":lon_ref}, method="linear")
map


<xarray.Dataset> Size: 8MB
Dimensions:  (time: 1, lat: 680, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 8B 2010-01-01
  * lat      (lat) float64 5kB -80.0 -79.75 -79.5 -79.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB -180.0 -179.7 -179.5 ... 179.4 179.7 179.9
Data variables:
    zos      (time, lat, lon) float64 8MB nan nan nan nan ... nan nan nan nan
Attributes:
    history:                   2024-10-23 12:55:06Z: Creation
    comment:                   Sea Surface Height measured by Altimetry and d...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    title:                     DT merged all satellites Global Ocean Gridded ...
    institution:               CLS, CNES
    references:                http://marine.copernicus.eu
    Conventions:               CF-1.6
    source:                    Altimetry measurements
    copernicusmarine_version:  2.0.1

In [4]:
# save data 
save_file="test_4th.nc"
# Sauvegarder le DataArray en fichier NetCDF
map.to_netcdf(save_file, unlimited_dims=["time"])

In [6]:
map

<xarray.Dataset> Size: 8MB
Dimensions:  (time: 1, lat: 680, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 8B 2010-01-01
  * lat      (lat) float64 5kB -80.0 -79.75 -79.5 -79.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB -180.0 -179.7 -179.5 ... 179.4 179.7 179.9
Data variables:
    zos      (time, lat, lon) float64 8MB nan nan nan nan ... nan nan nan nan
Attributes:
    history:                   2024-10-23 12:55:06Z: Creation
    comment:                   Sea Surface Height measured by Altimetry and d...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    title:                     DT merged all satellites Global Ocean Gridded ...
    institution:               CLS, CNES
    references:                http://marine.copernicus.eu
    Conventions:               CF-1.6
    source:                    Altimetry measurements
    copernicusmarine_version:  2.0.1

In [8]:
map['zos'] = map['zos'].astype(np.float32)

In [9]:
save_file="test_4th_float32.nc"
# Sauvegarder le DataArray en fichier NetCDF
map.to_netcdf(save_file, unlimited_dims=["time"])

In [63]:
# Import data to interp

file_map = "/Odyssey/private/t22picar/data/ssh_L4/SSH_L4_CMEMS_2010-01-01-2019-01-01.nc"
map = xr.open_dataset(file_map).sel(time=slice("2010-01-01","2010-01-01"))

map = map.rename({"longitude" : "lon"})
map = map.rename({"latitude" : "lat"})
map = map.rename({"adt" : "zos"})
#map

ds = map.interp({"lat":lat_ref, "lon":lon_ref}, method="linear")



start_date = datetime(2010, 1, 2)
end_date = datetime(2010, 1, 10)
current_date = start_date
time_index=1
# Boucle temporelle pour ajouter des données
while current_date < end_date:

    file_map = "/Odyssey/private/t22picar/data/ssh_L4/SSH_L4_CMEMS_2010-01-01-2019-01-01.nc"
    map = xr.open_dataset(file_map).sel(time=current_date)
    map = map.rename({"longitude" : "lon"})
    map = map.rename({"latitude" : "lat"})
    map = map.rename({"adt" : "zos"})
    map = map.interp({"lat":lat_ref, "lon":lon_ref}, method="linear")

    # Concaténation
    ds = xr.concat([ds, map], dim="time")
    current_date += timedelta(days=1)

In [64]:
ds

<xarray.Dataset> Size: 71MB
Dimensions:  (time: 9, lat: 680, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 72B 2010-01-01 2010-01-02 ... 2010-01-09
  * lat      (lat) float64 5kB -80.0 -79.75 -79.5 -79.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB -180.0 -179.7 -179.5 ... 179.4 179.7 179.9
Data variables:
    zos      (time, lat, lon) float64 71MB nan nan nan nan ... nan nan nan nan
Attributes:
    history:                   2024-10-23 12:55:06Z: Creation
    comment:                   Sea Surface Height measured by Altimetry and d...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    title:                     DT merged all satellites Global Ocean Gridded ...
    institution:               CLS, CNES
    references:                http://marine.copernicus.eu
    Conventions:               CF-1.6
    source:                    Altimetry measurements
    copernicusmarine_version:  2.0.1

In [65]:
# save data 
save_file="test_4th.nc"
# Sauvegarder le DataArray en fichier NetCDF
ds.to_netcdf(save_file)

In [40]:
from datetime import datetime, timedelta

with nc.Dataset(save_file, 'a') as dataset:
    print(dataset)
    start_date = datetime(2010, 1, 2)
    end_date = datetime(2010, 1, 3)
    current_date = start_date
    time_index=1
    # Boucle temporelle pour ajouter des données
    while current_date < end_date:
        # Calcul du temps (par exemple, en jours depuis une date de référence)
        var = dataset.variables['zos']
        time_var = dataset.variables['time']
        print(time_var)
        dernier_index = len(time_var) - 1
        print(dernier_index)
        
        file_map = "/Odyssey/private/t22picar/data/ssh_L4/SSH_L4_CMEMS_2010-01-01-2019-01-01.nc"
        map = xr.open_dataset(file_map).sel(time=current_date)
        map = map.rename({"longitude" : "lon"})
        map = map.rename({"latitude" : "lat"})
        map = map.rename({"adt" : "zos"})
        map = map.interp({"lat":lat_ref, "lon":lon_ref}, method="linear")
        ssh_i = map.zos.values

         # 4. Ajouter le nouveau temps à la variable 'time'
        time_var[dernier_index + 1] = nc.date2num(current_date, units=time_var.units, calendar=time_var.calendar)

        print(time_index)

        # Ajout des données dans le fichier NetCDF
        var[time_index, :, :] = ssh_i
        current_date += timedelta(days=1)
        time_index=time_index+1

        

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    history: 2024-10-23 12:55:06Z: Creation
    comment: Sea Surface Height measured by Altimetry and derived variables
    contact: servicedesk.cmems@mercator-ocean.eu
    title: DT merged all satellites Global Ocean Gridded SSALTO/DUACS Sea Surface Height L4 product and derived variables
    institution: CLS, CNES
    references: http://marine.copernicus.eu
    Conventions: CF-1.6
    source: Altimetry measurements
    copernicusmarine_version: 2.0.1
    dimensions(sizes): time(1), lat(680), lon(1440)
    variables(dimensions): float64 zos(time, lat, lon), float32 time(time), float64 lat(lat), float64 lon(lon)
    groups: 
<class 'netCDF4._netCDF4.Variable'>
float32 time(time)
    _FillValue: nan
    axis: T
    long_name: Time
    standard_name: time
    unit_long: Days Since 1950-01-01
    units: days since 1950-01-01
    calendar: gregorian
unlimited dimensions: 
current shape = (1,)
filling on
0

RuntimeError: NetCDF: Index exceeds dimension bound

In [4]:
map = map.interp({"lat":lat_ref, "lon":lon_ref}, method="linear")

In [4]:
map

<xarray.Dataset> Size: 8MB
Dimensions:  (lat: 680, lon: 1440)
Coordinates:
    time     datetime64[ns] 8B 2010-01-01
  * lat      (lat) float64 5kB -80.0 -79.75 -79.5 -79.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB -180.0 -179.7 -179.5 ... 179.4 179.7 179.9
Data variables:
    zos      (lat, lon) float64 8MB nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    history:                   2024-10-23 12:55:06Z: Creation
    comment:                   Sea Surface Height measured by Altimetry and d...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    title:                     DT merged all satellites Global Ocean Gridded ...
    institution:               CLS, CNES
    references:                http://marine.copernicus.eu
    Conventions:               CF-1.6
    source:                    Altimetry measurements
    copernicusmarine_version:  2.0.1

In [ ]:
# save data 
save_file=file_map[:-3]+"_4th"+".nc"
# Sauvegarder le DataArray en fichier NetCDF
map.to_netcdf(save_file)